# 通信会社 ネットワーク品質 AI エージェント ハンズオン

このハンズオンでは、Snowflake の AI 機能を使って**エリアマスタ、通信品質データ、設備キャパシティデータ、気象データ**の整備から **「自然言語で質問できる AI エージェント作成」** までを一気通貫で構築します。

### 今日つくるもの

携帯電話通信会社のネットワーク品質データ（通信速度・遅延・接続成功率・トラフィック量など）を横断的に分析できる AI エージェントです。  
完成すると、例えばこんな質問に自然言語で答えてくれます：

> 「2025年の東京エリアの通信速度推移を月別で可視化して。気温との相関も見たい」  
> 「現在設備が逼迫しているエリアと、そのトラフィック推移を教えて」  
> 「関東地方のエリア一覧とそれぞれの特徴を教えて」  
> 「雨の日と晴れの日で通信品質にどんな違いがある？」

---

### ハンズオンの流れ（所要時間：約80分）

| Step | やること | 学べる Snowflake 機能 | 所要時間 |
|:----:|----------|----------------------|:--------:|
| 1 | AI でエリアマスタを整備 | AI_CLASSIFY / AI_COMPLETE | 10分 |
| 2 | 気象データを取り込む | Snowflake Marketplace | 10分 |
| 3 | エリアの意味検索を可能にする | Cortex Search Service | 10分 |
| 4 | データの意味を定義する | Semantic View（Cortex Analyst） | 20分 |
| 5 | AI エージェントを組み立てる | Cortex Agent | 15分 |
| - | 動かしてみよう！ | - | 5分 |

## Step 0: 事前準備
使用するデータベース・スキーマ・ウェアハウスを設定します。

In [ ]:
USE DATABASE TELECOM_AI_HANDSON;
USE SCHEMA ANALYTICS;
USE WAREHOUSE COMPUTE_WH;

---
## Step 1: AI でエリアマスタを整備する

現在のエリアマスタ（`RAW.AREA_MASTER`）には、エリアID・エリア名・地方名しかありません。  
ここに Snowflake の **AI SQL 関数** を使って、2つの情報を自動で追加します。

### 追加する情報

#### 1. エリアタイプの自動分類 — `AI_CLASSIFY`

エリア名だけを見て、AI が自動的にエリアタイプを判定します。  
あらかじめ定義した **5つのタイプ** に分類します（MECE＝漏れなくダブりなく）。

| エリアタイプ | どんなエリア？ |
|-------------|---------------|
| 都心ビジネス | オフィス街、駅前商業地区、高層ビル密集地 |
| 商業・繁華街 | ショッピングモール、繁華街、娯楽施設集中地 |
| 住宅地 | マンション・戸建て住宅が中心の生活エリア |
| 観光・ウォーターフロント | 観光地、海辺、リゾート、歴史的エリア |
| 郊外・地方 | 工業地帯、農村部、山間地、人口密度が低いエリア |

#### 2. エリア特性説明文の自動生成 — `AI_COMPLETE`

エリア名と地方名から、そのエリアの **通信需要の特性や課題を説明する文章（約300文字）** を AI が自動で生成します。

> **ポイント**: `AI_CLASSIFY` はラベル＋説明文を渡すだけで分類してくれるので、従来のルールベースのロジックを書く必要がありません。

In [ ]:
SELECT * FROM TELECOM_AI_HANDSON.RAW.AREA_MASTER;

それでは実行しましょう。エリア数にもよりますが、**約1〜2分** で完了します。

In [ ]:
CREATE OR REPLACE TABLE TELECOM_AI_HANDSON.ANALYTICS.AI_AREA_MASTER AS
SELECT 
    AREA_ID,
    AREA_NAME,
    REGION_NAME,
    REGION_CODE,
    -- ① AI_CLASSIFY関数でエリアタイプを分類（MECE）
    AI_CLASSIFY(
        AREA_NAME,
        [
            {'label': '都心ビジネス', 'description': 'オフィス街、駅前商業地区、高層ビル密集地、ビジネス中心地'},
            {'label': '商業・繁華街', 'description': 'ショッピングモール、繁華街、娯楽施設集中地、歓楽街'},
            {'label': '住宅地', 'description': 'マンション・戸建て住宅が中心の生活エリア、ベッドタウン'},
            {'label': '観光・ウォーターフロント', 'description': '観光地、海辺、リゾート、歴史的エリア、臨海地区、湾岸'},
            {'label': '郊外・地方', 'description': '工業地帯、農村部、山間地、人口密度が低いエリア、地方都市郊外'}
        ],
        {'task_description': '日本の携帯電話通信エリア名から最も適切なエリアタイプに分類してください'}
    ):labels[0]::VARCHAR AS AREA_TYPE,
    -- ② AI_COMPLETE関数でエリア特性説明文生成
    TRIM(SNOWFLAKE.CORTEX.AI_COMPLETE(
        'llama3.1-405b',
        '以下の携帯電話通信エリアについて、通信需要の特性や課題を説明する文章（300文字程度）を日本語で1つ作成してください。
         エリア名: ' || AREA_NAME || '
         地方: ' || COALESCE(REGION_NAME, '不明') || '
         説明文:'
    )) AS AREA_DESCRIPTION,
    CURRENT_TIMESTAMP() AS CREATED_AT
FROM TELECOM_AI_HANDSON.RAW.AREA_MASTER;

### 結果を見てみましょう

AI が生成した `AREA_TYPE`（エリアタイプ）と `AREA_DESCRIPTION`（エリア特性説明文）を確認します。

In [ ]:
SELECT * FROM TELECOM_AI_HANDSON.ANALYTICS.AI_AREA_MASTER;

### エリアタイプ分類の精度を確認

5つのエリアタイプにバランスよく分類されているか確認しましょう。  
偏りがある場合は、`AI_CLASSIFY` に渡すエリアタイプの `description` を調整することで精度を改善できます。

In [ ]:
SELECT 
    AREA_TYPE,
    COUNT(*) AS AREA_COUNT
FROM TELECOM_AI_HANDSON.ANALYTICS.AI_AREA_MASTER
GROUP BY AREA_TYPE
ORDER BY AREA_COUNT DESC;

---
## Step 2: 気象データを取り込む（Snowflake Marketplace）

通信品質は天候に大きく左右されます。  
雨天時は電波の減衰が増し通信品質が低下、台風時はさらに顕著 — こうした **天候×通信品質の分析** を可能にするため、  
Snowflake Marketplace の **気象データ** を取り込みます。

### データソース

**Weather Source LLC: Frostbyte** が提供するグローバル気象データを使用します。  
日本（JP）の日次気象データを 2019年〜 取得できます。

### なぜテーブルを作るのか？

元データは華氏・インチ単位で、グローバルデータが含まれています。  
通信品質データと結合するには **日本のデータのみを抽出し、メートル法に変換して日付ごとに1行** にする必要があるため、  
全国平均に集約するテーブルを作成します。

| 集約カラム | 計算方法 | 説明 |
|------------|----------|------|
| 平均気温 | AVG | 全国の観測所の平均（℃に変換） |
| 最高気温 | MAX | 全国で最も高かった気温（℃に変換） |
| 最低気温 | MIN | 全国で最も低かった気温（℃に変換） |
| 降水量 | AVG | 全国平均の降水量（mmに変換） |
| 湿度 | AVG | 全国平均の湿度 |
| 日照時間 | AVG | 全国平均の日射量 |
| 降雪量 | SUM | 全国の降雪量合計（cmに変換） |
| 風速 | AVG | 全国平均の風速（km/hに変換） |

In [ ]:
CREATE OR REPLACE TABLE TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER AS
SELECT
    DATE_VALID_STD                                                      AS WEATHER_DATE,
    ROUND(AVG((AVG_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS AVG_TEMPERATURE,
    ROUND(MAX((MAX_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS MAX_TEMPERATURE,
    ROUND(MIN((MIN_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS MIN_TEMPERATURE,
    ROUND(AVG(TOT_PRECIPITATION_IN * 25.4), 1)                          AS AVG_RAINFALL,
    ROUND(AVG(AVG_HUMIDITY_RELATIVE_2M_PCT), 1)                         AS AVG_HUMIDITY,
    ROUND(AVG(AVG_RADIATION_SOLAR_TOTAL_WPM2), 2)                       AS AVG_SUNLIGHT_HOURS,
    ROUND(SUM(TOT_SNOWFALL_IN * 25.4), 1)                               AS TOTAL_SNOWFALL,
    ROUND(AVG(AVG_WIND_SPEED_10M_MPH * 1.60934), 1)                     AS AVG_WIND_SPEED
FROM WEATHER_SOURCE_LLC_FROSTBYTE.ONPOINT_ID.HISTORY_DAY
WHERE COUNTRY = 'JP'
GROUP BY DATE_VALID_STD
ORDER BY DATE_VALID_STD;

In [ ]:
SELECT * FROM TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER;

---
## Step 3: エリアの意味検索を可能にする（Cortex Search）

Step 1 で作った `AI_AREA_MASTER` に対して、**Cortex Search Service** を作成します。

### Cortex Search とは？

テキストの **意味** を理解して検索するサービスです。  
キーワードの完全一致ではなく、「こういう意味のものを探して」という検索ができます。

例えば「トラフィックが多い都市部のエリア」と聞くと、東京都心や大阪梅田などのエリアがヒットします。

### 仕組み

エリア名・地方名・エリアタイプ・AI説明文を **1つのテキストに結合** し、  
Snowflake の Embedding モデル（`snowflake-arctic-embed-l-v2.0`）でベクトル化して検索可能にします。

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE TELECOM_AI_HANDSON.ANALYTICS.AREA_SEARCH_SERVICE
    ON SEARCH_TEXT
    ATTRIBUTES AREA_ID, AREA_NAME, REGION_NAME, AREA_TYPE, AREA_DESCRIPTION
    WAREHOUSE = COMPUTE_WH
    TARGET_LAG = '1 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
    SELECT
        AREA_ID,
        AREA_NAME,
        REGION_NAME,
        REGION_CODE,
        AREA_TYPE,
        AREA_DESCRIPTION,
        CONCAT(
            COALESCE(AREA_NAME, ''), ' ',
            COALESCE(REGION_NAME, ''), ' ',
            COALESCE(AREA_TYPE, ''), ' ',
            COALESCE(AREA_DESCRIPTION, '')
        ) AS SEARCH_TEXT
    FROM TELECOM_AI_HANDSON.ANALYTICS.AI_AREA_MASTER
);

---
## Step 4: データの意味を定義する（セマンティックビュー）

ここがこのハンズオンの **キモ** です。

Cortex Analyst に「通信速度を教えて」と聞いたとき、どのテーブルの・どのカラムを・どう集計すればよいかを  
AI が理解できるように、**データの意味（セマンティクス）** を定義します。

### セマンティックビューで定義すること

| 定義項目 | 何を定義する？ | 具体例 |
|----------|---------------|--------|
| **テーブル** | 分析対象のテーブルとその説明 | NETWORK_QUALITY, EQUIPMENT_STATUS, WEATHER |
| **リレーションシップ** | テーブル間の結合条件 | 計測日 = 気象観測日 |
| **ファクト** | 集計対象の数値カラム | 通信速度、遅延、トラフィック量、気温 |
| **ディメンション** | 分析の切り口となる属性 | エリア名、地方、日付 |
| **メトリクス** | よく使う集計パターン | 平均ダウンロード速度＝AVG(下り速度) |
| **シノニム** | 日本語の別名・同義語 | 「通信速度」「下り速度」「ダウンロード速度」 |

### 3つのテーブルの関係

```
              AREA_ID              MEASURE_DATE = WEATHER_DATE
NETWORK_QUALITY ───────▶ EQUIPMENT_STATUS    NETWORK_QUALITY ────────────────▶ WEATHER
(通信品質)                (設備状況)           (通信品質)                        (気象)
```

> **ポイント**: シノニムを豊富に定義しておくと、「通信速度」でも「ダウンロード速度」でも「下り」でも同じカラムを参照してくれます。

In [ ]:
CREATE OR REPLACE SEMANTIC VIEW TELECOM_AI_HANDSON.ANALYTICS.TELECOM_ANALYST

    -- ========================================
    -- テーブル定義
    -- ========================================
    tables (
        NETWORK_QUALITY AS TELECOM_AI_HANDSON.ANALYTICS.NETWORK_QUALITY
            PRIMARY KEY (AREA_ID, MEASURE_DATE)
            WITH SYNONYMS = ('通信品質', 'ネットワーク品質', '品質データ', '通信データ', 'NW品質', '通信速度データ', 'KPI')
            COMMENT = '日次ネットワーク品質データ。2022-01-01〜2026-02-24のエリア別日次通信速度・遅延・接続成功率・トラフィック量を含む。',

        EQUIPMENT_STATUS AS TELECOM_AI_HANDSON.ANALYTICS.EQUIPMENT_STATUS
            PRIMARY KEY (AREA_ID)
            WITH SYNONYMS = ('設備', '設備状況', '設備データ', '基地局', 'キャパシティ', '設備キャパシティ', '負荷')
            COMMENT = '設備キャパシティスナップショット（2026-02-24時点）。エリア別の基地局数・最大容量・負荷率・設備ステータスを含む。',

        WEATHER AS TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER
            PRIMARY KEY (WEATHER_DATE)
            WITH SYNONYMS = ('気象', '天気', '気象データ', '天気データ', '気温', '降水量', '天候', '気象庁', '気象情報')
            COMMENT = '全国日次平均気象データ（2015年〜）。気象庁の全観測所データを日別に集約した全国平均の気温・降水量・湿度・日照時間・降雪量・風速。'
    )

    -- ========================================
    -- リレーションシップ定義
    -- ========================================
    relationships (
        NQ_EQUIPMENT AS NETWORK_QUALITY(AREA_ID) REFERENCES EQUIPMENT_STATUS(AREA_ID),
        NQ_WEATHER AS NETWORK_QUALITY(MEASURE_DATE) REFERENCES WEATHER(WEATHER_DATE)
    )

    -- ========================================
    -- ファクト定義（数値データ）
    -- ========================================
    facts (
        -- 通信品質ファクト
        NETWORK_QUALITY.download_speed_fact AS AVG_DOWNLOAD_MBPS
            COMMENT = '平均ダウンロード速度（Mbps）。下り通信速度。',
        NETWORK_QUALITY.upload_speed_fact AS AVG_UPLOAD_MBPS
            COMMENT = '平均アップロード速度（Mbps）。上り通信速度。',
        NETWORK_QUALITY.latency_fact AS AVG_LATENCY_MS
            COMMENT = '平均遅延（ミリ秒）。レイテンシ。',
        NETWORK_QUALITY.connection_success_fact AS CONNECTION_SUCCESS_RATE
            COMMENT = '接続成功率（%）。100%に近いほど良好。',
        NETWORK_QUALITY.call_drop_fact AS CALL_DROP_RATE
            COMMENT = '切断率（%）。0%に近いほど良好。',
        NETWORK_QUALITY.packet_loss_fact AS PACKET_LOSS_RATE
            COMMENT = 'パケットロス率（%）。0%に近いほど良好。',
        NETWORK_QUALITY.traffic_fact AS TOTAL_TRAFFIC_GB
            COMMENT = '日次総トラフィック量（GB）',

        -- 設備ファクト
        EQUIPMENT_STATUS.base_station_count_fact AS BASE_STATION_COUNT
            COMMENT = '基地局数（局）',
        EQUIPMENT_STATUS.max_capacity_fact AS MAX_CAPACITY_GBPS
            COMMENT = '最大通信容量（Gbps）',
        EQUIPMENT_STATUS.current_load_fact AS CURRENT_LOAD_PCT
            COMMENT = '現在の設備負荷率（%）。100%に近いほど逼迫。',

        -- 気象ファクト
        WEATHER.avg_temperature_fact AS AVG_TEMPERATURE
            COMMENT = '全国日平均気温（℃）。全観測所の平均。',
        WEATHER.max_temperature_fact AS MAX_TEMPERATURE
            COMMENT = '全国日最高気温（℃）。全観測所の最大値。',
        WEATHER.min_temperature_fact AS MIN_TEMPERATURE
            COMMENT = '全国日最低気温（℃）。全観測所の最小値。',
        WEATHER.avg_rainfall_fact AS AVG_RAINFALL
            COMMENT = '全国日平均降水量（mm）',
        WEATHER.avg_humidity_fact AS AVG_HUMIDITY
            COMMENT = '全国日平均湿度（%）',
        WEATHER.avg_sunlight_hours_fact AS AVG_SUNLIGHT_HOURS
            COMMENT = '全国日平均日照時間（時間）',
        WEATHER.total_snowfall_fact AS TOTAL_SNOWFALL
            COMMENT = '全国日降雪量合計（cm）',
        WEATHER.avg_wind_speed_fact AS AVG_WIND_SPEED
            COMMENT = '全国日平均風速（km/h）'
    )

    -- ========================================
    -- ディメンション定義（属性データ）
    -- ========================================
    dimensions (
        -- ========== NETWORK_QUALITY ディメンション ==========
        NETWORK_QUALITY.area_id AS AREA_ID
            WITH SYNONYMS = ('エリアID', 'エリアコード', '基地局エリア', 'エリア識別子')
            COMMENT = 'エリア識別コード（例：AREA_KT_001）',
        NETWORK_QUALITY.area_name AS AREA_NAME
            WITH SYNONYMS = ('エリア名', 'エリア', '地域名', '地区名', '基地局エリア名')
            COMMENT = '通信エリア名。東京都心北部エリア、大阪梅田エリアなどを含む。',
        NETWORK_QUALITY.region_name AS REGION_NAME
            WITH SYNONYMS = ('地方', '地方名', 'リージョン', '地域')
            COMMENT = '地方名（関東、近畿、東海、九州）',
        NETWORK_QUALITY.measure_date AS MEASURE_DATE
            WITH SYNONYMS = ('計測日', '測定日', '日付', '対象日')
            COMMENT = '通信品質を計測した日付（DATE型、2022-01-01〜2026-02-24）',
        NETWORK_QUALITY.measure_year AS YEAR(MEASURE_DATE)
            WITH SYNONYMS = ('計測年', '年', '対象年')
            COMMENT = '計測年（2022〜2026）',
        NETWORK_QUALITY.measure_month AS MONTH(MEASURE_DATE)
            WITH SYNONYMS = ('計測月', '月', '対象月')
            COMMENT = '計測月（1〜12）',

        -- ========== EQUIPMENT_STATUS ディメンション ==========
        EQUIPMENT_STATUS.area_id AS AREA_ID
            WITH SYNONYMS = ('エリアID', 'エリアコード')
            COMMENT = 'エリア識別コード',
        EQUIPMENT_STATUS.area_name AS AREA_NAME
            WITH SYNONYMS = ('エリア名', 'エリア')
            COMMENT = 'エリア名',
        EQUIPMENT_STATUS.region_name AS REGION_NAME
            WITH SYNONYMS = ('地方', '地方名')
            COMMENT = '地方名',
        EQUIPMENT_STATUS.snapshot_date AS SNAPSHOT_DATE
            WITH SYNONYMS = ('スナップショット日', '基準日', '設備確認日')
            COMMENT = '設備データの取得日（2026-02-24）',
        EQUIPMENT_STATUS.equipment_status AS EQUIPMENT_STATUS
            WITH SYNONYMS = ('設備ステータス', '設備状態', '設備区分', 'ステータス', '負荷状態')
            COMMENT = '設備の負荷状態を表すステータス（正常 / 注意 / 逼迫）',

        -- ========== WEATHER ディメンション ==========
        WEATHER.weather_date AS WEATHER_DATE
            WITH SYNONYMS = ('観測日', '気象日', '天気の日付', '気象観測日')
            COMMENT = '気象データの観測日（DATE型、2015年〜）。NETWORK_QUALITYのMEASURE_DATEとリレーションで紐づく。',
        WEATHER.weather_year AS YEAR(WEATHER_DATE)
            WITH SYNONYMS = ('観測年', '気象年')
            COMMENT = '気象データの観測年',
        WEATHER.weather_month AS MONTH(WEATHER_DATE)
            WITH SYNONYMS = ('観測月', '気象月')
            COMMENT = '気象データの観測月（1〜12）'
    )

    -- ========================================
    -- メトリクス定義（集計関数）
    -- ========================================
    metrics (
        -- 通信品質メトリクス
        NETWORK_QUALITY.avg_download_speed AS AVG(NETWORK_QUALITY.download_speed_fact)
            WITH SYNONYMS = ('平均ダウンロード速度', '平均下り速度', '下り平均', '平均通信速度')
            COMMENT = 'ダウンロード速度の平均（Mbps）',
        NETWORK_QUALITY.avg_upload_speed AS AVG(NETWORK_QUALITY.upload_speed_fact)
            WITH SYNONYMS = ('平均アップロード速度', '平均上り速度', '上り平均')
            COMMENT = 'アップロード速度の平均（Mbps）',
        NETWORK_QUALITY.avg_latency AS AVG(NETWORK_QUALITY.latency_fact)
            WITH SYNONYMS = ('平均遅延', '平均レイテンシ', 'レイテンシ平均')
            COMMENT = '遅延の平均（ms）',
        NETWORK_QUALITY.avg_connection_success AS AVG(NETWORK_QUALITY.connection_success_fact)
            WITH SYNONYMS = ('平均接続成功率', '接続成功率平均', '接続率')
            COMMENT = '接続成功率の平均（%）',
        NETWORK_QUALITY.avg_call_drop AS AVG(NETWORK_QUALITY.call_drop_fact)
            WITH SYNONYMS = ('平均切断率', '切断率平均', '通話切断率')
            COMMENT = '切断率の平均（%）',
        NETWORK_QUALITY.avg_packet_loss AS AVG(NETWORK_QUALITY.packet_loss_fact)
            WITH SYNONYMS = ('平均パケットロス率', 'パケロス平均', 'ロス率')
            COMMENT = 'パケットロス率の平均（%）',
        NETWORK_QUALITY.total_traffic AS SUM(NETWORK_QUALITY.traffic_fact)
            WITH SYNONYMS = ('総トラフィック', '合計トラフィック', 'トラフィック合計', '総通信量')
            COMMENT = 'トラフィック量の合計（GB）',
        NETWORK_QUALITY.avg_daily_traffic AS AVG(NETWORK_QUALITY.traffic_fact)
            WITH SYNONYMS = ('日平均トラフィック', '日次平均トラフィック', '平均日次通信量')
            COMMENT = '日次トラフィック量の平均（GB）',

        -- 設備メトリクス
        EQUIPMENT_STATUS.total_base_stations AS SUM(EQUIPMENT_STATUS.base_station_count_fact)
            WITH SYNONYMS = ('総基地局数', '合計基地局数', '基地局数合計')
            COMMENT = '基地局数の合計（局）',
        EQUIPMENT_STATUS.total_capacity AS SUM(EQUIPMENT_STATUS.max_capacity_fact)
            WITH SYNONYMS = ('総容量', '合計容量', '最大容量合計')
            COMMENT = '最大通信容量の合計（Gbps）',
        EQUIPMENT_STATUS.avg_load AS AVG(EQUIPMENT_STATUS.current_load_fact)
            WITH SYNONYMS = ('平均負荷率', '負荷率平均', '平均設備負荷')
            COMMENT = '設備負荷率の平均（%）',

        -- 気象メトリクス
        WEATHER.period_avg_temperature AS AVG(WEATHER.avg_temperature_fact)
            WITH SYNONYMS = ('平均気温', '気温平均', '平均温度')
            COMMENT = '期間内の全国平均気温（℃）',
        WEATHER.period_max_temperature AS MAX(WEATHER.max_temperature_fact)
            WITH SYNONYMS = ('最高気温', '最高温度', '期間最高気温')
            COMMENT = '期間中の全国最高気温（℃）',
        WEATHER.period_min_temperature AS MIN(WEATHER.min_temperature_fact)
            WITH SYNONYMS = ('最低気温', '最低温度', '期間最低気温')
            COMMENT = '期間中の全国最低気温（℃）',
        WEATHER.period_avg_rainfall AS AVG(WEATHER.avg_rainfall_fact)
            WITH SYNONYMS = ('平均降水量', '降水量平均')
            COMMENT = '期間内の全国平均降水量（mm）',
        WEATHER.period_avg_humidity AS AVG(WEATHER.avg_humidity_fact)
            WITH SYNONYMS = ('平均湿度', '湿度平均')
            COMMENT = '期間内の全国平均湿度（%）',
        WEATHER.period_avg_sunlight AS AVG(WEATHER.avg_sunlight_hours_fact)
            WITH SYNONYMS = ('平均日照時間', '日照時間平均')
            COMMENT = '期間内の全国平均日照時間（時間）',
        WEATHER.period_total_snowfall AS SUM(WEATHER.total_snowfall_fact)
            WITH SYNONYMS = ('総降雪量', '合計降雪量', '累積降雪量')
            COMMENT = '期間内の全国降雪量合計（cm）',
        WEATHER.period_avg_wind_speed AS AVG(WEATHER.avg_wind_speed_fact)
            WITH SYNONYMS = ('平均風速', '風速平均')
            COMMENT = '期間内の全国平均風速（km/h）'
    )

    COMMENT = '携帯電話通信会社のネットワーク品質・設備・気象分析のためのセマンティックビュー。関東・近畿・東海・九州の50エリアの通信品質データを含む。2022年〜の日次通信品質データ（下り/上り速度、遅延、接続成功率、切断率、パケットロス率、トラフィック量）、2026-02-24時点の設備キャパシティスナップショット（基地局数、最大容量、負荷率）、および気象庁の全国日次平均気象データ（気温・降水量・湿度・日照時間・降雪量等）を統合。計測日と気象観測日がリレーションで紐づき、天候と通信品質の相関分析が可能。';

### セマンティックビューの確認

正しく定義できたか確認しましょう。テーブル・ファクト・ディメンション・メトリクスの一覧が表示されます。

In [ ]:
DESCRIBE SEMANTIC VIEW TELECOM_ANALYST;

---
## Step 5: AI エージェントを組み立てる（Cortex Agent）

いよいよ最後のステップです。  
ここまで作ってきた **Cortex Search** と **Cortex Analyst** を組み合わせて、  
1つの **AI エージェント** にまとめます。

### エージェントの仕組み

```
ユーザーの質問
     │
     ▼
┌─────────────────────────┐
│   Telecom Agent        │
│  （オーケストレーター）   │
│                        │
│  質問の意図を判断して      │
│  適切なツールを選択        │
└────┬──────────┬─────────┘
     │          │
     ▼          ▼
┌─────────┐ ┌──────────────┐
│ Cortex  │ │   Cortex     │
│ Search  │ │  Analyst     │
│         │ │              │
│エリア情報│  │通信品質・設備・│
│の意味検索│ │気象の数値分析  │
└─────────┘ └──────────────┘
```

### ツールの使い分け

| 質問の種類 | 使われるツール | 例 |
|-----------|---------------|----|
| エリアの特徴・情報を知りたい | **Cortex Search** | 「関東のエリア一覧を教えて」 |
| 数値の集計・分析をしたい | **Cortex Analyst** | 「2025年の月別通信速度は？」 |
| 両方が必要 | **Search → Analyst** | 「逼迫エリアのトラフィック推移は？」 |

In [ ]:
CREATE OR REPLACE AGENT TELECOM_AI_HANDSON.ANALYTICS.TELECOM_AGENT
    COMMENT = '携帯電話通信会社のネットワーク品質・設備・気象分析とエリア検索を行うエージェント'
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": ""
  },
  "instructions": {
    "response": "あなたは携帯電話通信会社のネットワーク品質アナリストです。ユーザーからの質問に対して、通信品質データ（下り/上り速度、遅延、接続成功率、切断率、パケットロス率、トラフィック量）、設備キャパシティデータ（基地局数、最大容量、負荷率）、気象データを活用して回答してください。数値データは適切にフォーマットして表示してください（例：速度はMbps表記、トラフィックはGB表記、比率は%表記）。エリア名は正確に記載してください。分析結果には根拠となるデータを含めてください。日本語で丁寧に回答してください。可能であれば視覚化を提供してください。時系列のトレンドは線グラフをデフォルトとし、エリア比較は棒グラフを使用してください。天候と通信品質の関係（雨天時の品質低下、気温とトラフィック量の相関など）にも注意して分析してください。",
    "orchestration": "エリアの詳細情報（エリア名、地方、エリアタイプ、AI説明文など）を検索する場合はcortex searchを使用し、通信品質・設備・気象の数値分析にはcortex analystを使用してください。複合的な質問の場合は、まずcortex searchでエリアを特定し、その結果をcortex analystに渡して分析してください。気温や天候と通信品質の関係を聞かれた場合は、cortex analystでネットワーク品質データと気象データを組み合わせて分析してください。",
    "sample_questions": [
      {
        "question": "2025年の東京都心エリアの下り通信速度推移を月別で可視化してください。気温との相関も見たいです。"
      },
      {
        "question": "トラフィック量が最も多い月はいつですか？どのエリアが特にトラフィックが集中しますか？"
      },
      {
        "question": "現在設備が逼迫しているエリアを教えてください。それぞれのエリアの過去のトラフィック推移も見せてください。"
      },
      {
        "question": "関東地方のエリア一覧とそれぞれの特徴を教えてください。"
      },
      {
        "question": "2024年と2025年でトラフィックが大きく増加したエリアトップ5を教えてください。"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "TELECOM_ANALYST",
        "description": "携帯電話通信会社のネットワーク品質データ、設備キャパシティデータ、全国日次気象データをクエリして分析します。下り/上り通信速度、遅延、接続成功率、切断率、パケットロス率、トラフィック量の集計や、気温・降水量と通信品質の相関分析に使用します。2022年〜2026年2月の日次通信品質データ、2026年2月時点の設備キャパシティスナップショット、2015年〜の全国平均気象データを含みます。"
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "AREA_SEARCH",
        "description": "携帯電話通信エリアのマスタからエリア情報を検索します。エリア名、地方名、AIが分類したエリアタイプ（都心ビジネス、商業・繁華街、住宅地、観光・ウォーターフロント、郊外・地方）、AIが生成したエリア特性説明文などのテキスト検索に使用します。"
      }
    }
  ],
  "tool_resources": {
    "TELECOM_ANALYST": {
      "semantic_view": "TELECOM_AI_HANDSON.ANALYTICS.TELECOM_ANALYST"
    },
    "AREA_SEARCH": {
      "name": "TELECOM_AI_HANDSON.ANALYTICS.AREA_SEARCH_SERVICE",
      "max_results": 10
    }
  }
}
$$;

### エージェントの確認

In [ ]:
SHOW AGENTS IN SCHEMA TELECOM_AI_HANDSON.ANALYTICS;

---
## 動かしてみよう！

エージェントが完成しました！Snowsight の **Cortex Agent UI** から以下のような質問を試してみましょう。

### 試してみたい質問例

**通信品質分析**
- 「2025年の東京都心エリアの下り通信速度推移を月別で可視化してください。気温との相関も見たいです。」
- 「2024年と2025年でトラフィックが大きく増加したエリアトップ5を教えてください。」

**天候×通信品質**
- 「降水量が多い日と少ない日で通信速度にどんな違いがありますか？」
- 「台風シーズン（8-9月）の通信品質への影響を分析してください。」

**設備管理**
- 「現在設備が逼迫しているエリアを教えてください。それぞれの過去のトラフィック推移も見せてください。」
- 「設備負荷率が注意レベルのエリアはどこですか？」

**エリア検索**
- 「関東地方のエリア一覧とそれぞれの特徴を教えてください。」
- 「観光地エリアの通信品質の傾向を教えてください。」

**Web検索**
- 「2026年に予定されている5G展開計画について教えてください。」
- 「最近の通信障害ニュースはありますか？」